# **미니프로젝트 4차 1대1 문의 내용 유형 분류기**
# 단계3 : Text classification

### 문제 정의
> 1:1 문의 내용 분류 문제<br>
> 1. 문의 내용 분석
> 2. 문의 내용 분류 모델 성능 평가
### 학습 데이터
> * 1:1 문의 내용 데이터 : train.csv

### 변수 소개
> * text : 문의 내용
> * label : 문의 유형

### References
> * Machine Learning
>> * [sklearn-tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
> * Deep Learning
>> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
>> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
>> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)
>> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)

## 1. 개발 환경 설정

### 1-1. 라이브러리 설치

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn gensim wordcloud python-mecab-ko wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.6/575.6 KB 32.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 KB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 38.2 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=d00f60a4621cb8e5058385c33419b573a065867a1a00afbecf9a6617d9bdd5f4
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


### 1-2. 라이브러리 import

In [2]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wget,os
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import wget,os

### 1-3. 한글 글꼴 설정(Windows)

In [3]:
# if not os.path.exists("malgun.ttf"): 
#     wget.download("https://www.wfonts.com/download/data/2016/06/13/malgun-gothic/malgun.ttf")
# if 'malgun' not in fm.fontManager.findfont("Malgun Gothic"):
#     fm.fontManager.addfont("malgun.ttf")
# if plt.rcParams['font.family']!= ["Malgun Gothic"]:
#     plt.rcParams['font.family']= [font for font in fm.fontManager.ttflist if 'malgun.ttf' in font.fname][-1].name
# plt.rcParams['axes.unicode_minus'] = False #한글 폰트 사용시 마이너스 폰트 깨짐 해결
# assert plt.rcParams['font.family'] == ["Malgun Gothic"], "한글 폰트가 설정되지 않았습니다."
# FONT_PATH = "malgun.ttf"

In [ ]:
# !sudo apt-get install -y fonts-nanum

### 1-4. 자바 경로 설정(Windows)

In [ ]:
# os.environ['JAVA_HOME'] = "C:\Program Files\Java\jdk-19"

### 1-3. 한글 글꼴 설정(Colab)

In [3]:
!sudo apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-525
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 1s (8,268 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

In [4]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
print(font_name)
plt.rcParams['font.family']=font_name
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


### 1-4. 구글드라이브 연결(Colab)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 전처리한 데이터 불러오기
* 1, 2일차에 전처리한 데이터를 불러옵니다.
* sparse data에 대해서는 scipy.sparse.load_npz 활용

In [6]:
train = pd.read_csv('/content/drive/MyDrive/2023/AIVLE/NLP미프/2023.04.03_미니프로젝트4차_실습자료/train.csv')

In [7]:
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

preprocessed_df = train.replace({'label' : label_dict}).copy()

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(preprocessed_df['text'], preprocessed_df['label'], test_size=0.3, random_state=42)

In [20]:
import re 

def clean_text(texts): 
    corpus = [] 
    for i in range(0, len(texts)): 

        review = re.sub(r'[@%\\*=()/~#&\+á?\xc3\xa1\-\|\.\:\;\!\-\,\_\~\$\'\"\n\>\<]', '',texts[i]) #@%*=()/+ 와 같은 문장부호 제거 ]\[\은 예외처리
        review = re.sub(r'\d+','', review)#숫자 제거
        review = review.lower() #소문자 변환
        review = re.sub(r'\s+', ' ', review) #extra space 제거
        review = re.sub(r'<[^>]+>','',review) #Html tags 제거
        review = re.sub(r'\s+', ' ', review) #spaces 제거
        review = re.sub(r"^\s+", '', review) #space from start 제거
        review = re.sub(r'\s+$', '', review) #space from the end 제거
        review = re.sub(r'_', ' ', review) #space from the end 제거
        corpus.append(review) 
        
    return corpus

In [21]:
X_train = clean_text(X_train.values)
X_val = clean_text(X_val.values)

In [14]:
from konlpy.tag import Okt 

han_sentence = "오늘도 열심히 코딩을 해볼까요? 같이 힘내서 자연어 처리 고수가 됩시다! ㅎㅎ"
okt = Okt() # 인스턴스 할당
print("한국어 형태소 분석 결과(어간 추출X) ==>", okt.morphs(han_sentence, stem = False)) # 형태소 단위로 분리
print("한국어 형태소 분석 결과(어간 추출O) ==>", okt.morphs(han_sentence, stem = True)) # 형태소 단위로 분리 후 어간 추출

한국어 형태소 분석 결과(어간 추출X) ==> ['오늘', '도', '열심히', '코딩', '을', '해볼까', '요', '?', '같이', '힘내서', '자연어', '처리', '고수', '가', '됩시다', '!', 'ㅎㅎ']
한국어 형태소 분석 결과(어간 추출O) ==> ['오늘', '도', '열심히', '코딩', '을', '해보다', '요', '?', '같이', '힘내다', '자연어', '처리', '고수', '가', '되다', '!', 'ㅎㅎ']


In [20]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in X_train: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
X_train = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.
X_train = X_train[0]
#train.head() # 데이터 확인

In [21]:
tokenized = [] # 데이터프레임의 한 컬럼으로 추가할 리스트
for sentence in X_val: # 전처리된 리뷰들을 하나씩 꺼내옵니다
    tokens = okt.morphs(sentence, stem = True) # 형태소 분석 (stem = True로 설정해 어간 추출을 해주었습니다)
    tokenize = " ".join(tokens) # tokens라는 리스트 안의 형태소들을 띄어쓰기로 분리된 하나의 문자열로 join시켜줍니다.
    tokenized.append(tokenize) # 형태소 단위로 띄어쓰기된 문자열을 최종 리스트에 추가해줍니다
X_val = pd.DataFrame(tokenized) # 리스트를 데이터프레임으로 변환해 tokenized_stem라는 컬럼명으로 추가해줍니다.
X_val = X_val[0]
#train.head() # 데이터 확인

## 3. Machine Learning(N-grams)
* N-gram으로 전처리한 데이터를 이용하여 3개 이상의 Machine Learning 모델 학습 및 성능 분석
> * [sklearn-tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Vectorization 적용하여 학습 데이터셋과 테스트 데이터 셋 변환. 
tfidf_vect = TfidfVectorizer(ngram_range=(1,1),  min_df = 1, max_df=300,analyzer = 'word', sublinear_tf = False)
tfidf_vect.fit(X_train)

X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_val_tfidf_vect = tfidf_vect.transform(X_val) # train셋으로 fit한 벡터라이저 이용해 transform
print('학습 & 테스트 데이터 Text의 TfidfVectorizer Shape:',X_train_tfidf_vect.shape, X_val_tfidf_vect.shape)

학습 & 테스트 데이터 Text의 TfidfVectorizer Shape: (2594, 8868) (1112, 8868)


### 3-1. Model 1

In [39]:
from sklearn.linear_model import LogisticRegression #모델 불러오기
from sklearn import metrics

# LogisticRegression을 이용하여 학습/예측/평가 수행. 
lr_clf = LogisticRegression(solver='liblinear', C = 10) 
lr_clf.fit(X_train_tfidf_vect , y_train)
pred = lr_clf.predict(X_val_tfidf_vect)
print('TF-IDF Logistic Regression 의 예측 정확도는 {0:.3f}'.format(metrics.accuracy_score(y_val ,pred)))

TF-IDF Logistic Regression 의 예측 정확도는 0.803


### 3-2. Model 2

### 3-3. Model 3

### 3-4. Hyperparameter Tuning(Optional) 
* Manual Search, Grid search, Bayesian Optimization, TPE...
> * [grid search tutorial sklearn](https://scikit-learn.org/stable/modules/grid_search.html)
> * [optuna tutorial](https://optuna.org/#code_examples)
> * [ray-tune tutorial](https://docs.ray.io/en/latest/tune/examples/tune-sklearn.html)

## 4. Deep Learning(Sequence)
* Sequence로 전처리한 데이터를 이용하여 DNN, 1-D CNN, LSTM 등 3가지 이상의 deep learning 모델 학습 및 성능 분석
> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)

### 4-1. DNN

In [9]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [23]:
str_len_max = np.max(X_train.str.len()) # 리뷰 길이의 최대값 계산
print('최대 길이 :',round(str_len_max))

최대 길이 : 6051


In [60]:
str_len_mean = np.mean(X_train.str.len()) # 리뷰 길이의 평균값 계산
print('최대 길이 :',round(str_len_mean))

AttributeError: ignored

In [64]:
max_words = 3987 ## 위에서 40,000으로 설정함
embedding_dim = 128 ## 단어 embedding 지원. 단어 하나당 몇 개의 특징값을 학습할 것인가
max_len = 80 ## 문장 최대 길이

In [65]:
### Tokenizer here
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=max_words, lower=False)  # lower (대문자->소문자) 옵션은 한국어를 할땐 끄자.

In [25]:
# Text --> Sequence
%%time
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_val = tokenizer.texts_to_sequences(X_val)

CPU times: user 279 ms, sys: 1.97 ms, total: 281 ms
Wall time: 280 ms


In [26]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
#### Pad Sequences here
X_train = pad_sequences(X_train, maxlen = max_len)
X_val = pad_sequences(X_val, maxlen = max_len)

In [63]:
X_train.shape

(2594, 40)

In [29]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.backend import clear_session
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Conv1D, Bidirectional, LSTM, GRU, RNN, MaxPool1D, Flatten

In [30]:
X_train.shape[1]

40

In [31]:
#####################
# 1. 세션 초기화
clear_session()

# 2. 모델 사슬처럼 엮기
il = Input(shape=(X_train.shape[1],))

# 1. 임베딩 레이어 : 임베딩차원은 128
el = Embedding(max_words,
               embedding_dim,
               input_length=max_len)(il)

# 2. Conv1D 블록 : 필터수 64개, 윈도우 사이즈 5
hl1 = Conv1D(filters=64,
             kernel_size=5,
             activation='swish')(el)
# 3. Bidirectional layer :
#     * 정방향 : LSTM, 히든스테이트 32 
#     * 역방향 : LSTM, 히든스테이트 32
lstm32 = LSTM(32, return_sequences=True)
hl2 = Bidirectional(lstm32)(hl1)
# 4. Bidirectional layer :
#     * 정방향 : GRU, 히든스테이트 32
#     * 역방향 : RNN, 히든스테이트 16
forward_gru32 = GRU(32, return_sequences=True)
backward_lstm16 = LSTM(16, return_sequences=True, go_backwards=True)
hl3 = Bidirectional(forward_gru32, backward_layer=backward_lstm16)(hl2)
# 5. Conv1D 블록 : 필터수 32개, 윈도우 사이즈 5
hl4 = Conv1D(filters=32,
             kernel_size=5,
             activation='swish')(hl3)
# 6. MaxPool1D 블록 : 필터사이즈2
hl5 = MaxPool1D(pool_size=2)(hl4)
# 7. 플래튼
hl6 = Flatten()(hl5)
# 8. FC Layer : 노드 1024개
hl7 = Dense(1024, activation='relu')(hl6)
# 9. 시그모이드 레이어
ol = Dense(5, activation='softmax')(hl7)

# 3. 모델 처음과 끝 지정
model = Model(il, ol)

# 4. 컴파일
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics='accuracy')

# 요약
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 40)]              0         
                                                                 
 embedding (Embedding)       (None, 40, 128)           510336    
                                                                 
 conv1d (Conv1D)             (None, 36, 64)            41024     
                                                                 
 bidirectional (Bidirectiona  (None, 36, 64)           24832     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 36, 48)           14592     
 nal)                                                            
                                                                 
 conv1d_1 (Conv1D)           (None, 32, 32)            7712  

* Using pre-trained word Embedding  
* 남이 사용한 임베딩을 가져와 임베딩하기
* GloVe, Word2Vec 등...

# EarlyStopping을 이용한 학습.

1. 20%는 벨리데이션 셋.
2. 4epochs전과 비교하여 early stopping할 것.

In [32]:
from tensorflow.keras.callbacks import EarlyStopping

In [33]:
#####################
es = EarlyStopping(monitor='val_loss',
                   min_delta=0,
                   patience=10,
                   verbose=1,
                   restore_best_weights=True)

In [34]:
from tensorflow.keras.utils import to_categorical

y_train_ctg = to_categorical(y_train)
y_val_ctg = to_categorical(y_val)

In [35]:
y_train_ctg.shape

(2594, 5)

In [36]:
X_train.shape

(2594, 40)

In [37]:
y_train_ctg

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [38]:
#####################
model.fit(X_train, y_train_ctg, epochs=30, verbose=1, callbacks=[es], validation_split=0.2)

Epoch 1/30
65/65 [==============================] - 35s 185ms/step - loss: 1.1935 - accuracy: 0.4935 - val_loss: 1.0631 - val_accuracy: 0.5645
Epoch 2/30
65/65 [==============================] - 6s 88ms/step - loss: 0.8068 - accuracy: 0.6487 - val_loss: 0.9964 - val_accuracy: 0.5607
Epoch 3/30
65/65 [==============================] - 5s 81ms/step - loss: 0.5194 - accuracy: 0.8019 - val_loss: 0.9807 - val_accuracy: 0.6802
Epoch 4/30
65/65 [==============================] - 3s 45ms/step - loss: 0.2641 - accuracy: 0.9065 - val_loss: 1.1024 - val_accuracy: 0.7052
Epoch 5/30
65/65 [==============================] - 3s 41ms/step - loss: 0.1652 - accuracy: 0.9393 - val_loss: 1.3674 - val_accuracy: 0.7129
Epoch 6/30
65/65 [==============================] - 2s 37ms/step - loss: 0.1087 - accuracy: 0.9711 - val_loss: 1.4624 - val_accuracy: 0.7245
Epoch 7/30
65/65 [==============================] - 2s 26ms/step - loss: 0.0737 - accuracy: 0.9754 - val_loss: 1.7029 - val_accuracy: 0.7110
Epoch 7: ea

In [39]:
y_pred = model.predict(X_val)

35/35 [==============================] - 2s 16ms/step


In [42]:
np.argmax(y_pred, axis=1)

array([1, 2, 2, ..., 3, 1, 0])

In [59]:
from sklearn.metrics import classification_report


y = y_val
p = np.argmax(y_pred, axis=1)

target_names = ['코드1,2(0)', '웹(1)', '이론(2)', '시스템운영(3)', '원격(4)']

label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}

preprocessed_df = train.replace({'label' : label_dict}).copy()

print(classification_report(y, p, 
                            target_names=target_names))

              precision    recall  f1-score   support

    코드1,2(0)       0.78      0.85      0.81       462
        웹(1)       0.50      0.54      0.52       217
       이론(2)       0.52      0.59      0.55       239
    시스템운영(3)       0.78      0.43      0.55       162
       원격(4)       0.56      0.31      0.40        32

    accuracy                           0.66      1112
   macro avg       0.63      0.54      0.57      1112
weighted avg       0.66      0.66      0.65      1112



In [54]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support

## performance metrics
accuracy = accuracy_score(y, p)

precision, recall, fscore, support = \
    precision_recall_fscore_support(y, p)

print('Accuracy   : %.3f' %accuracy) # (102+164)/(102+16+3+164)
print('Precision  : %.3f' %precision[0]) # 102/(102+3)
print('Recall     : %.3f' %recall[0]) # 102/(102+16)
print('Specificyty: %.3f' %recall[1]) # 164/(3+164)
print('F1-Score   : %.3f' %fscore[0]) # 2/(1/precision + 1/recall) = 2/(1/0.971+1/0.864)


Accuracy   : 0.656
Precision  : 0.781
Recall     : 0.848
Specificyty: 0.544
F1-Score   : 0.813


### 4-2. 1-D CNN

### 4-3. LSTM

## 5. Using pre-trained model(Optional)
* 한국어 pre-trained model로 fine tuning 및 성능 분석
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)
> * [HuggingFace-Korean](https://huggingface.co/models?language=korean)